In [1]:
import os

In [2]:
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file 

'c:\\ProjectML\\ProjectML_with_MLFlow\\research'

In [3]:
os.chdir("../")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../") 

In [4]:
%pwd

'c:\\ProjectML\\ProjectML_with_MLFlow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mahendra867/ProjectML_with_MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mahendra867"
os.environ["MLFLOW_TRACKING_PASSWORD"]="85969b2c9b582440861229562a757d53c3cbb020"

### creating the entity with class 

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path  # this root_dir,test_data_path,model_path,metric_file_name are the variables takes from the model_evaluation configuration code part of config.yaml file 
    test_data_path: Path
    model_path: Path
    all_params: dict # these all params is available inside the params.yaml file which it reads all the parameters 
    metric_file_name: Path
    target_column: str # this target column takes from schema.yaml file 
    mlflow_uri: str # this mlflow uri iagain write it down once again inside mlflow URI section 


### creating the configuration manager src 

In [7]:
from PROJECTML.constants import *
from PROJECTML.utils.common import read_yaml, create_directories, save_json

In [8]:
# this template is same for every stage like data_ingestion, data_validation,data_transformation,model_trainer,model_evaluation

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    # only this part of code get changes her actually iam defining a get_model_evaluation_config
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation# here our model ElasticNet reading all params , Target_column of schema 
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params, # here iam saving all the parameters which i do experment while getting to know at which parameters our trained model which is ElastcNet is doing well
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/mahendra867/ProjectML_with_MLFlow.mlflow", # here i have setted the mlflow URI
           
        )

        return model_evaluation_config # so which ever the varaibles we initilized inside the entity those are all getting returned 


### creating components of model evaluation

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score # here i have imported all the evaluation meterics for evaluating our model peformance
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:

# here i have defined one ModelEvaluation class in which i have defined the ModelEvaluationConfig
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    # here i have defiend the eval_metrics which it evaluates our model performance by calculating the rmse,mae,r2
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

# here i have defined one fucntion called log_into_mlflow which it will mlflow 
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path) # here iam loading the test dataset
        model = joblib.load(self.config.model_path) # here iam loading my trained model 

        test_x = test_data.drop([self.config.target_column], axis=1) # here iam dropping my targetcolumn in test_x
        test_y = test_data[[self.config.target_column]] # here iam keeping my target column


        mlflow.set_registry_uri(self.config.mlflow_uri) # here iam setting my tracking URI it is getting from configuration manager src because i want to everything inside a remote server 
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x) # here iam predicting my testing data

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities) # here it is calculating my metrics of the predicted model
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2} # these scores are saving inside my artifacts of model_evaluation as a jason file
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params) # here iam logging with mlflow as log params,different log_mertics , and here iam giving all the parameters together which it will save inside ml flow

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

# here it will check whether we are doing inside the URI or in local directory , i already set the URL so it will save inside the URI 
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel") # here iam giving model ElasticnetModel
            else:
                mlflow.sklearn.log_model(model, "model")

    


### creating a pipeline of model evalaution

In [11]:
try:
    config = ConfigurationManager() # here i have initlized my ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config() # her eiam getiing my get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config) # here iam passing this  model_evaluation_config to my ModelEvalaution
    model_evaluation_config.log_into_mlflow() # here iam starting this log_into_mlflow()
except Exception as e: # if anything goes wrong it throws the exception
    raise e

[2023-12-14 11:16:32,763: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-14 11:16:32,763: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-14 11:16:32,774: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-14 11:16:32,774: INFO: common: created directory at: artifacts]
[2023-12-14 11:16:32,781: INFO: common: created directory at: artifacts/model_evaluation]


[2023-12-14 11:16:34,613: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/12/14 11:16:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
